In [ ]:
# inspired from https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=

In [1]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [14]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np
import nltk
import re
import random
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from time import time
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import unidecode


C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
raw_metadata = pd.read_csv("C:/projects/machineLearning/hecCourse/metadata.csv")

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
def selectRandomArticles(num, data):
    articleListIndex = []
    for _ in range(0, num):
        articleListIndex.append(random.randint(1,len(data)))
    return articleListIndex



C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
NAs = pd.isna(raw_metadata['abstract']) 
metadata = raw_metadata[np.invert(NAs)]
print(len(metadata))

# Reset les index metadata
metadata = metadata.reset_index(drop=True)

# Extract ID and abstract from dataset


indexRandom = selectRandomArticles(500, metadata)


df_abstracts = metadata[['cord_uid', 'abstract']][indexRandom]  #Nombre d'abstranct sélectionnés


def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text


df_abstracts['abstract'] = df_abstracts.apply(lambda x: remove_accented_chars(x['abstract']), axis=1) # convert the accents into non accents in each

# Créer une liste des abstract nettoyé des chiffre et caractères spéciaux
list_abs = []
for i in df_abstracts['abstract']:

    list_abs.append(re.sub(r'[^a-zA-Z]+',' ',i)) # !!! Modifier pour ne pas retirer les accents
    #list_abs.append(i)
print("Nombre d'article retenus: ",len(list_abs))
 

print(list_abs[0])
print(metadata['abstract'][0])


C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
214463


TypeError: selectRandomArticles() got an unexpected keyword argument 'numArticles'

In [18]:
stemmer = SnowballStemmer('english')
# Ajouter un stemmer à la fonction "CountVectorizer" :
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
scv = StemmedCountVectorizer(strip_accents = 'ascii'# Retirer les accents sur les lettres
                             ,lowercase = True      # Retirer les majuscules 
                             ,stop_words='english'  # Retirer les STOPWORDS à aprtir d'une liste
                             #,stop_words= my_stopwords_list  # Retirer les STOPWORDS à aprtir d'une liste
                             ,analyzer='word'
                             #,min_df=2  # Élimine les mots qui apparaissent dans moins de 2 documents
                             ,max_df=0.70  # Élimine les mots qui apparaissent dans plus de 75% des documents
                             )

# 2- Créer la matrice "count_matrix" avec le décompte de chaque mot par document
count_matrix = scv.fit_transform(list_abs)


tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 

# 4- Calcul des valeurs IDF:
tfidf_transformer.fit(count_matrix)

# 4- Créer la matrice de score TF-IDF
tfidf_matrix = tfidf_transformer.transform(count_matrix)

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
count_matrix

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<500x6362 sparse matrix of type '<class 'numpy.int64'>'
	with 41128 stored elements in Compressed Sparse Row format>

In [38]:
# for simple count matrix
lda_tf = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf.fit(count_matrix)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tfidf.fit(tfidf_matrix)

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=8, random_state=0)

In [39]:
p = pyLDAvis.sklearn.prepare(lda_tf, count_matrix, scv)

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
pyLDAvis.sklearn.prepare(lda_tf, count_matrix, scv)

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.165593  0.056403       1        1  30.171869
1      0.066430 -0.069840       2        1  22.186017
2      0.031627 -0.093155       3        1  15.166162
0     -0.121887 -0.058052       4        1  12.989997
5      0.092591  0.008108       5        1   9.302113
4      0.056700  0.170032       6        1   5.383147
3      0.040132 -0.013497       7        1   4.800695, topic_info=           Term        Freq       Total Category  logprob  loglift
4173    patient  252.000000  252.000000  Default  30.0000  30.0000
6084     vaccin  265.000000  265.000000  Default  29.0000  29.0000
852        cell  446.000000  446.000000  Default  28.0000  28.0000
2870  influenza  378.000000  378.000000  Default  27.0000  27.0000
6189      virus  552.000000  552.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
4906    respons   10.308578  202.390919   Topic7  -5.6838   0.0592
4824     region    8.938011   90.402945   Topic7  -5.8265   0.7224
4912     result    9.130925  298.355609   Topic7  -5.8051  -0.4502
5491      studi    8.730430  291.088952   Topic7  -5.8500  -0.4704
287     antigen    8.512578   95.807169   Topic7  -5.8752   0.6156

[523 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4         2  0.889950         ab
52        1  0.024157       acid
52        2  0.591847       acid
52        3  0.048314       acid
52        4  0.024157       acid
...     ...       ...        ...
6218      3  0.881074        vsv
6236      5  0.820126       wasp
6286      1  0.982781     winter
6287      6  0.829197  wintertim
6303      1  0.969696     worker

[1137 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 3, 1, 6, 5, 4])

In [35]:
pyLDAvis.save_html(p, 'ldaSimpleCountMatrix.html')

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
p2 = pyLDAvis.sklearn.prepare(lda_tfidf, tfidf_matrix, scv)

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
pyLDAvis.sklearn.prepare(lda_tfidf, tfidf_matrix, scv)

C:\Users\franc\.virtualenvs\machineLearningProject-YX0hXYec\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.137650  0.000486       1        1  77.128904
1     -0.029195  0.013125       2        1   4.735144
2     -0.023931 -0.004730       3        1   3.929307
0     -0.023456 -0.003803       4        1   3.887087
3     -0.021687 -0.003698       5        1   3.647160
4     -0.021742 -0.002272       6        1   3.609839
5     -0.017638  0.000892       7        1   3.062559, topic_info=            Term      Freq     Total Category  logprob  loglift
1988     extract  0.000000  0.000000  Default  30.0000  30.0000
4360       plant  1.000000  1.000000  Default  29.0000  29.0000
1152      conjug  0.000000  0.000000  Default  28.0000  28.0000
5380      splice  0.000000  0.000000  Default  27.0000  27.0000
4783       recod  0.000000  0.000000  Default  26.0000  26.0000
...          ...       ...       ...      ...      ...      ...
2456        hbov  0.071479  0.442646   Topic7  -7.2868   1.6626
246       anemia  0.054447  0.321979   Topic7  -7.5590   1.7087
725           ca  0.067179  0.503183   Topic7  -7.3489   1.4723
82    adenovirus  0.064328  0.556372   Topic7  -7.3922   1.3285
4526         prf  0.053662  0.345965   Topic7  -7.5735   1.6223

[274 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
67        1  0.901688       activ
155       1  1.287985      airway
408       1  1.061432      associ
513       1  1.016411        base
798       1  1.005148        case
852       1  0.983040        cell
1115      1  1.349917    compound
1152      1  1.345356      conjug
1363      1  1.049471        data
1484      1  0.967378     develop
1553      1  1.014813      diseas
1706      1  0.954905      effect
1978      1  0.940479     express
1988      1  1.030160     extract
2260      1  1.036271        gene
2467      1  0.988403      health
2560      1  1.024099        high
2655      1  1.018800       human
2754      1  0.893707       immun
2860      1  0.985657      infect
2870      1  0.944695   influenza
2941      1  0.992788  interferon
3623      1  0.952454       model
3949      1  0.912452        nurs
4117      1  0.950528      pandem
4173      1  0.992176     patient
4360      1  0.775212       plant
4610      1  1.027258     protein
4906      1  0.954410     respons
4912      1  0.911970      result
4977      1  0.985437         rna
5159      1  0.976352     sequenc
5380      1  1.268250      splice
5491      1  1.051158       studi
6072      1  0.994854         use
6084      1  1.014505      vaccin
6130      1  1.069447      ventil
6174      1  0.910172       viral
6189      1  1.014632       virus, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 3, 1, 4, 5, 6])

In [ ]:
pyLDAvis.save_html(p, 'ldatfIdfCountMatrix.html')